## Факторизационная машина второго порядка

In [7]:
import numpy as np
import time
from scipy.sparse import *
from sklearn.externals import joblib
import pandas as pd

In [3]:
def RMSE(y_pred, y_true):
    n = len(y_true)
    loss = np.subtract(y_true, y_pred)
    return np.sqrt(np.sum((loss)**2) / n)

def R_2(y_pred, y_true):
    loss = np.subtract(y_true, y_pred)
    return 1 - np.sum((loss)**2) / np.sum((y_true - np.mean(y_true))**2) 

class FactorizationMachine(object):
    def __init__(self, step=0.000001, k=5):
        self.k = k
        self.w0 = 0
        self.W = None #d,1
        self.V = None #d,k
        self.Z = None
        self.X2 = None
        self.initStep = step
        self.step = self.initStep
        self.loss_history = []

    def predict(self, X):
        self.Z = X @ self.V #n,k
        if issparse(X):
            self.X2 = X.power(2)
        else:
            self.X2 = X**2
        return self.w0 + X @ self.W + np.sum(self.Z**2 - (self.X2 @ self.V**2), axis=1, keepdims=True) / 2

    def get_batches(self, dataset, batch_size):
        X, Y = dataset
        n_samples = X.shape[0]

        # Shuffle at the start of epoch
        indices = np.arange(n_samples)
        np.random.shuffle(indices)

        for start in range(0, n_samples, batch_size):
            end = min(start + batch_size, n_samples)
            batch_idx = indices[start:end]

            yield X[batch_idx], Y[batch_idx]

    def addLoss(self, predictions, y_true):
        self.loss_history.append(RMSE(predictions.reshape(-1), y_true))

    def updatedtLearningRate(self, epoch):
        #magicConst = 200
        #self.step = self.initStep / np.log(np.log(np.log(epoch + magicConst)))
        self.step = self.initStep / np.sqrt(epoch + 1)
        

    def fit(self, X, y, n_epoch=30, batch_size=128):
        n, d = X.shape

        # This is a nice initialization
        stdv = 1. / np.sqrt(n)
        self.W = np.random.uniform(-stdv, stdv, size=(d, 1))
        self.V = np.random.uniform(-stdv, stdv, size=(d, self.k))
    
        for i in range(n_epoch):
            for x_batch, y_batch in self.get_batches((X, y), batch_size):
                self.updatedtLearningRate(i)
                predictions = self.predict(x_batch)
                dLoss = 2 * np.subtract(predictions, y_batch.reshape(-1, 1)) / len(y_batch)

                self.w0 -= np.multiply(self.step, np.sum(dLoss))
                
                self.W -= np.multiply(self.step, x_batch.transpose() @ dLoss)

                np.multiply(self.step, dLoss, out=dLoss)
                for j in range(self.k):
                    if issparse(x_batch):
                        dV = x_batch.multiply(self.Z[:, j].reshape(-1, 1))
                        dV -= self.X2.multiply(self.V[:, j])
                    else:
                        dV = np.multiply(x_batch, self.Z[:, j].reshape(-1, 1))
                        np.subtract(dV, np.multiply(self.X2, self.V[:, j]), out=dV)
                    self.V[:,j] -= dLoss.reshape(-1) @ dV
                    
            self.addLoss(predictions, y_batch)
        return self.loss_history

## Тестирование на синтетических данных

In [4]:
from sklearn.externals import joblib
X_t2 = joblib.load('X.bin')
y_t2 = joblib.load('y.bin')

<b>Parameters:</b> скорость обучения log

In [475]:
fm = FactorizationMachine(step=0.000003, k=8)
loss_history = fm.fit(X_t2, y_t2, n_epoch=300, batch_size=256)
print('loss_history', loss_history[-10:])
print('shape', X_t2.shape, y_t2.shape)
pr = fm.predict(X_t2)

loss_history [0.639181729560903, 0.7192722521497317, 0.6214670865891395, 0.5448647235097059, 0.8620369373732033, 0.8978984266486023, 0.7812319952096726, 0.5801703746665489, 0.5789855014009749, 0.9489588462589356]
shape (10000, 7) (10000, 1)


In [423]:
reg = SGDRegressor(fit_intercept=True, penalty='none', shuffle=False, eta0=0.001, max_iter=100)
reg.fit(X_t2, y_t2)
prLR = reg.predict(X_t2)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [452]:
pr

array([[ -7.84309394],
       [-12.77575163],
       [ 87.7635759 ],
       ...,
       [-73.02295607],
       [ 55.44797985],
       [-64.43883501]])

In [424]:
prLR

array([ -7.82540695, -12.74182483,  87.74340881, ..., -73.05613334,
        55.46920415, -64.30510889])

In [420]:
y_t2

array([[ -7.82540687],
       [-12.74182486],
       [ 87.74340893],
       ...,
       [-73.05613324],
       [ 55.4692042 ],
       [-64.30510887]])

## Тестирование на миниданных

### Подготовка small dataset

In [5]:
import pandas as pd

In [4]:
df = pd.read_csv('dataset-small-ratings.csv')
df = df[['userId', 'movieId', 'rating']]

In [5]:
df[0:5]

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [5]:
y = df['rating'].values
y

array([4., 4., 4., ..., 5., 5., 3.])

In [6]:
X = df[['userId', 'movieId']].values
X

array([[     1,      1],
       [     1,      3],
       [     1,      6],
       ...,
       [   610, 168250],
       [   610, 168252],
       [   610, 170875]])

In [7]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=True)
x = onehot_encoder.fit_transform(X)
print(x.shape)

(100836, 10334)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [19]:
from sklearn.externals import joblib
joblib.dump(x, 'small_x.bin')

['small_x.bin']

In [46]:
joblib.dump(y, 'small_y.bin')

['small_y.bin']

### Загрузка минидатасета 

In [140]:
x = joblib.load('small_x.bin')
type(x), x.shape
y = joblib.load('small_y.bin')

### Обучение с целью подборки параметров обучения
<b>Parameters:</b> coeff learing rate = 0.1, dims factors - k = 2

In [37]:
fm_model = FactorizationMachine(step=0.1, k=2) #0.1
loss_history = fm_model.fit(x, y, n_epoch=100, batch_size=256)
print('loss_histoy', loss_history)
prRating = fm_model.predict(x)

loss_histoy [16.359718842662826, 16.626196059992314, 17.081824947956537, 18.284834311170105, 16.858090113179916, 17.441897588369407, 16.390528805052373, 16.418083632953966, 16.540941083792298, 16.97579980816147, 17.768894742882043, 17.377312549649254, 17.852421882982956, 18.239388220328816, 16.694025699997546, 17.19496358324938, 16.20112388578001, 18.455525799727628, 17.55934340056728, 17.909277280429222, 18.50537854558733, 17.11412139830555, 16.127202195976462, 17.644472210432124, 17.907725360617018, 18.641666662537503, 16.878470699844875, 17.941989423635622, 18.064980509697307, 17.30988333879418, 18.410853077117565, 17.74351792862676, 17.655421766396348, 17.83286645107326, 18.374447302155637, 17.607731536490565, 19.297023420142402, 18.307181219380897, 17.37963252406058, 17.879633055811365, 18.250388021810032, 18.039150476850033, 18.320667125252463, 18.227029750983167, 17.97760128546723, 16.38717379603662, 17.343540794856892, 17.12448002511052, 18.01019310894371, 18.325871256641257, 1

In [36]:
prRating.shape,y.shape, prRating[:5], prRating[-5:]

((100836, 1), (100836,), array([[4.65006635],
        [4.07197704],
        [4.67408587],
        [4.73883132],
        [4.9876469 ]]), array([[3.61905183],
        [3.76522243],
        [3.6881375 ],
        [3.99411645],
        [3.53720915]]))

In [20]:
y[:5], y[-5:],

(array([4., 4., 4., 5., 5.]), array([4., 5., 5., 5., 3.]))

In [39]:
score = RMSE(prRating.reshape(-1), y)
score

0.8155666332048287

In [38]:
score_R2 = R_2(prRating.reshape(-1), y)
score_R2

0.3880066296442618

### Кроссвалидация
<b>Dataset:</b> Small dataset

<b> С параметрами: </b> 
coef learing rate = 0.007, n_epoch = 20, k = 4, func learing rate - "log", batch_size = 256

In [41]:
from sklearn.model_selection import KFold #
kf = KFold(n_splits=3) # Define the split - into 3 folds 
kf.get_n_splits(x) # returns the number of splitting iterations in the cross-validator
RMSE_history_tr = []
R_2_history_tr = []
RMSE_history_ts = []
R_2_history_ts = []

print(kf)
counter = 0
for train_index, test_index in kf.split(x):
    counter += 1
    print('fold:', counter)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    fm_mdl = FactorizationMachine(step=0.007, k=4)
    fm_mdl.fit(x, y, n_epoch=20, batch_size=256)

    predictions_test = fm_mdl.predict(X_train).reshape(-1)
    RMSE_history_tr.append(RMSE(predictions_test, y_train))
    R_2_history_tr.append(R_2(predictions_test, y_train))
    
    predictions_test = fm_mdl.predict(X_test).reshape(-1)
    RMSE_history_ts.append(RMSE(predictions_test, y_test))
    R_2_history_ts.append(R_2(predictions_test, y_test))

KFold(n_splits=3, random_state=None, shuffle=False)
fold: 1
fold: 2
fold: 3


In [46]:
metricsNames = ['R2-train','R2-test', 'RMSE-train', 'RMSE-test']
colNames = ['T1', 'T2', 'T3']
#report =  pd.DataFrame([], index = metricsNames)
report = pd.DataFrame(np.vstack([R_2_history_tr, R_2_history_ts, RMSE_history_tr, RMSE_history_ts]), index=metricsNames, 
                     columns=colNames)
E = pd.Series(report.mean(axis=1), index = report.index)
STD = pd.Series(report.std(axis=1), index = report.index)
report['E'] = E
report['STD'] = STD

report

,T1,T2,T3,E,STD
R2-train,0.177422,0.170949,0.162223,0.170198,0.007627
R2-test,0.151557,0.170501,0.183038,0.168365,0.015849
RMSE-train,0.944844,0.955545,0.947535,0.949308,0.005566
RMSE-test,0.958268,0.936703,0.952876,0.949283,0.011223


### Кроссвалидация
<b>Dataset:</b> Small dataset

<b> С параметрами: </b> 
coeff learing rate = 0.1, n_epoch = 20, k = 2, func learing rate - "log", batch_size = 256

In [42]:
from sklearn.model_selection import KFold #
kf = KFold(n_splits=3) # Define the split - into 3 folds 
kf.get_n_splits(x) # returns the number of splitting iterations in the cross-validator
RMSE_history_tr = []
R_2_history_tr = []
RMSE_history_ts = []
R_2_history_ts = []

print(kf)
counter = 0
for train_index, test_index in kf.split(x):
    counter += 1
    print('fold:', counter)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    fm_mdl = FactorizationMachine(step=0.1, k=2)
    fm_mdl.fit(x, y, n_epoch=20, batch_size=256)

    predictions_test = fm_mdl.predict(X_train).reshape(-1)
    RMSE_history_tr.append(RMSE(predictions_test, y_train))
    R_2_history_tr.append(R_2(predictions_test, y_train))
    
    predictions_test = fm_mdl.predict(X_test).reshape(-1)
    RMSE_history_ts.append(RMSE(predictions_test, y_test))
    R_2_history_ts.append(R_2(predictions_test, y_test))

KFold(n_splits=3, random_state=None, shuffle=False)
fold: 1
fold: 2
fold: 3


In [43]:
metricsNames = ['R2-train','R2-test', 'RMSE-train', 'RMSE-test']
colNames = ['T1', 'T2', 'T3']
#report =  pd.DataFrame([], index = metricsNames)
report = pd.DataFrame(np.vstack([R_2_history_tr, R_2_history_ts, RMSE_history_tr, RMSE_history_ts]), index=metricsNames, 
                     columns=colNames)
E = pd.Series(report.mean(axis=1), index = report.index)
STD = pd.Series(report.std(axis=1), index = report.index)
report['E'] = E
report['STD'] = STD

report

,T1,T2,T3,E,STD
R2-train,0.329671,0.319887,0.314330,0.321296,0.007767
R2-test,0.301327,0.325146,0.333368,0.319947,0.016641
RMSE-train,0.852935,0.865468,0.857212,0.858538,0.006371
RMSE-test,0.869587,0.844887,0.860754,0.858409,0.012516


## New: 
### Кроссвалидация
<b>Dataset:</b> Small dataset

<b> С параметрами: </b> 
coeff learing rate = 0.9, n_epoch = 80, k = 3, func learing rate - "sqrt", batch_size = 1024

In [142]:
from sklearn.model_selection import KFold #
kf = KFold(n_splits=3) # Define the split - into 3 folds 
kf.get_n_splits(x) # returns the number of splitting iterations in the cross-validator
RMSE_history_tr = []
R_2_history_tr = []
RMSE_history_ts = []
R_2_history_ts = []

print(kf)
counter = 0
for train_index, test_index in kf.split(x):
    counter += 1
    print('fold:', counter)
    X_train, X_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    fm_mdl = FactorizationMachine(step=0.9, k=3)
    fm_mdl.fit(x, y, n_epoch=80, batch_size=1024)

    predictions = fm_mdl.predict(X_train).reshape(-1)
    RMSE_history_tr.append(RMSE(predictions, y_train))
    R_2_history_tr.append(R_2(predictions, y_train))
    
    predictions = fm_mdl.predict(X_test).reshape(-1)
    RMSE_history_ts.append(RMSE(predictions, y_test))
    R_2_history_ts.append(R_2(predictions, y_test))

KFold(n_splits=3, random_state=None, shuffle=False)
fold: 1
fold: 2
fold: 3


In [143]:
metricsNames = ['R2-train','R2-test', 'RMSE-train', 'RMSE-test']
colNames = ['T1', 'T2', 'T3']
report = pd.DataFrame(np.vstack([R_2_history_tr, R_2_history_ts, RMSE_history_tr, RMSE_history_ts]), index=metricsNames, 
                     columns=colNames)
E = pd.Series(report.mean(axis=1), index = report.index)
STD = pd.Series(report.std(axis=1), index = report.index)
report['E'] = E
report['STD'] = STD

report

,T1,T2,T3,E,STD
R2-train,0.327637,0.318525,0.312609,0.319590,0.007570
R2-test,0.299644,0.323296,0.331310,0.318083,0.016464
RMSE-train,0.854228,0.866334,0.858288,0.859617,0.006161
RMSE-test,0.870634,0.846045,0.862081,0.859586,0.012483


## Тестирование на полном датасете

### Подготовка small dataset

In [47]:
df = pd.read_csv('dataset-netflixSorted.csv')


In [48]:
df = df[['User', 'Movie', 'Rate']]

In [49]:
df[:10]

,User,Movie,Rate
0,510180,1367,5
1,510180,1798,5
2,510180,2866,3
3,510180,3730,4
4,510180,3870,2
5,1086,295,4
6,1086,682,4
7,1086,829,3
8,1086,1700,3
9,122223,607,4


In [50]:
target = df['Rate'].values
target
joblib.dump(target, 'netflix_y.bin')

['netflix_y.bin']

In [51]:
feats = df[['User', 'Movie']].values
feats
joblib.dump(feats, 'netflix_denseX.bin')

['netflix_denseX.bin']

In [52]:
onehot_encoder = OneHotEncoder(sparse=True)
F = onehot_encoder.fit_transform(feats)
print(F.shape)
joblib.dump(F, 'netflix_sparseX.bin')

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(24053764, 475257)


['netflix_sparseX.bin']

### Загрузка датасета Full Netflix

In [4]:
F = joblib.load('netflix_sparseX.bin')
target = joblib.load('netflix_y.bin')
print('Feats', type(F), F.shape, 'targets', type(target), target.shape)

Feats <class 'scipy.sparse.csr.csr_matrix'> (24053764, 475257) targets <class 'numpy.ndarray'> (24053764,)


### Обучение с целью подборки параметров обучения

<b>Parameters:</b> coeff learing rate = 0.1, dims factors - k = 2, batch_size = 256, func learning rate - "log"

In [64]:
fm_model = FactorizationMachine(step=0.1, k=2)
t_start = time.time()
loss_history = fm_model.fit(F, target, n_epoch=1, batch_size=256)
t_run = time.time() - t_start 
print('run time: m', t_run / 60)
print('loss_histoy', len(loss_history), loss_history)


run time: m 15.48952515522639
loss_histoy 1 [1.5324512690356045]


In [69]:
predictions = fm_model.predict(F).reshape(-1)
predictions.shape


(24053764,)

In [70]:
rmse = RMSE(predictions, target)
rms

1.0074915491681806

## New:
<b>Parameters:</b> coeff learing rate = 0.9, dims factors - k = 3, batch_size = 1024, func learning rate - "sqrt"

In [145]:
fm_model = FactorizationMachine(step=0.9, k=3)
t_start = time.time()
loss_history = fm_model.fit(F, target, n_epoch=5, batch_size=1024)
t_run = time.time() - t_start 
print('run time: m', t_run / 60)
print('loss_histoy', len(loss_history), loss_history)

run time: m 25.97569557825724
loss_histoy 5 [0.4459035271345388, 1.0278057483100784, 1.204134754594045, 0.5452592278819187, 1.1041529634864515]


In [147]:
predictions = fm_model.predict(F).reshape(-1)
predictions.shape

(24053764,)

In [148]:
r2 = R_2(predictions, target)
r2

0.21294889469047595

In [149]:
rmse = RMSE(predictions, target)
rmse

0.9635595228463429

In [153]:
fm_model = FactorizationMachine(step=0.9, k=3)
t_start = time.time()
loss_history = fm_model.fit(F, target, n_epoch=7, batch_size=1024)
t_run = time.time() - t_start 
print('run time: m', t_run / 60)
print('loss_histoy', len(loss_history), loss_history)

run time: m 35.608677037556966
loss_histoy 7 [0.6459725238652039, 0.6041741031190966, 0.8811284125199534, 1.9437431604974107, 0.7641688506818161, 0.9228448986100676, 1.5332854709011514]


In [154]:
predictions = fm_model.predict(F).reshape(-1)
predictions.shape

(24053764,)

In [155]:
r2 = R_2(predictions, target)
r2

0.18289598330597046

In [156]:
rmse = RMSE(predictions, target)
rmse

0.9817835565158282

In [158]:
fm_model = FactorizationMachine(step=0.9, k=3)
t_start = time.time()
loss_history = fm_model.fit(F, target, n_epoch=5, batch_size=1024)
t_run = time.time() - t_start 
print('run time: m', t_run / 60)
print('loss_histoy', len(loss_history), loss_history)

run time: m 25.60951056877772
loss_histoy 5 [0.903924584356503, 0.7947459110598262, 0.7255890082089634, 0.8367849686433111, 1.2226057104974188]


In [159]:
predictions = fm_model.predict(F).reshape(-1)
predictions.shape

(24053764,)

In [160]:
r2 = R_2(predictions, target)
r2

0.09959997713551894

In [161]:
rmse = RMSE(predictions, target)
rmse

1.030611134058499

### Кроссвалидация 
<b>Dataset:</b> Full Netflix 

<b>Parameters:</b> coeff learing rate = 0.1, dims factors - k = 2, func learning rate - "log", batch_size = 256

In [88]:
from sklearn.model_selection import KFold #
kf = KFold(n_splits=3) # Define the split - into 3 folds 
kf.get_n_splits(F) # returns the number of splitting iterations in the cross-validator
RMSE_history_tr = []
RMSE_history_ts = []

print(kf)
counter = 0
for train_index, test_index in kf.split(F):
    counter += 1
    print('fold:', counter)
    X_train, X_test = F[train_index], F[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    fm_mdl = FactorizationMachine(step=0.1, k=2)
    fm_mdl.fit(X_train, y_train, n_epoch=1, batch_size=256)

    predictions = fm_mdl.predict(X_train).reshape(-1)
    RMSE_history_tr.append(RMSE(predictions, y_train))
    
    predictions = fm_mdl.predict(X_test).reshape(-1)
    RMSE_history_ts.append(RMSE(predictions, y_test))

KFold(n_splits=3, random_state=None, shuffle=False)
fold: 1
fold: 2
fold: 3


In [89]:
metricsNames = ['RMSE-train', 'RMSE-test']
colNames = ['T1', 'T2', 'T3']
report = pd.DataFrame(np.vstack([RMSE_history_tr, RMSE_history_ts]), index=metricsNames, 
                     columns=colNames)
E = pd.Series(report.mean(axis=1), index = report.index)
STD = pd.Series(report.std(axis=1), index = report.index)
report['E'] = E
report['STD'] = STD

report

,T1,T2,T3,E,STD
RMSE-train,0.997153,1.000739,0.977809,0.991900,0.012334
RMSE-test,1.070398,0.989470,1.036188,1.032018,0.040625


## New:
### Кроссвалидация 
<b>Dataset:</b> Full Netflix 

<b>Parameters:</b> coeff learing rate = 0.9, dims factors - k = 3, func learning rate - "sqrt", batch_size = 1024 

In [151]:
from sklearn.model_selection import KFold #
kf = KFold(n_splits=3) # Define the split - into 3 folds 
kf.get_n_splits(F) # returns the number of splitting iterations in the cross-validator
RMSE_history_tr = []
R_2_history_tr = []
RMSE_history_ts = []
R_2_history_ts = []

print(kf)
counter = 0
for train_index, test_index in kf.split(F):
    counter += 1
    print('fold:', counter)
    X_train, X_test = F[train_index], F[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    fm_mdl = FactorizationMachine(step=0.9, k=3)
    fm_mdl.fit(X_train, y_train, n_epoch=7, batch_size=1024)

    predictions = fm_mdl.predict(X_train).reshape(-1)
    RMSE_history_tr.append(RMSE(predictions, y_train))
    R_2_history_tr.append(R_2(predictions, y_train))
    
    predictions = fm_mdl.predict(X_test).reshape(-1)
    RMSE_history_ts.append(RMSE(predictions, y_test))
    R_2_history_ts.append(R_2(predictions, y_test))

KFold(n_splits=3, random_state=None, shuffle=False)
fold: 1
fold: 2
fold: 3


In [152]:
metricsNames = ['R2-train','R2-test', 'RMSE-train', 'RMSE-test']
colNames = ['T1', 'T2', 'T3']
report = pd.DataFrame(np.vstack([R_2_history_tr, R_2_history_ts, RMSE_history_tr, RMSE_history_ts]), index=metricsNames, 
                     columns=colNames)
E = pd.Series(report.mean(axis=1), index = report.index)
STD = pd.Series(report.std(axis=1), index = report.index)
report['E'] = E
report['STD'] = STD

report

,T1,T2,T3,E,STD
R2-train,-0.077084,0.234834,0.249195,0.135648,0.184371
R2-test,0.022666,0.150605,0.097379,0.090217,0.064270
RMSE-train,1.107705,0.963869,0.940394,1.003989,0.090584
RMSE-test,1.096353,0.968829,1.029015,1.031399,0.063796


## NEW:

<b>Dataset:</b> Full Netflix 

<b>Parameters:</b> coeff learing rate = 0.9, dims factors - k = 3, func learning rate - "sqrt", batch_size = 1024 

In [5]:
from sklearn.model_selection import KFold #
kf = KFold(n_splits=3) # Define the split - into 3 folds 
kf.get_n_splits(F) # returns the number of splitting iterations in the cross-validator
RMSE_history_tr = []
R_2_history_tr = []
RMSE_history_ts = []
R_2_history_ts = []

print(kf)
counter = 0
for train_index, test_index in kf.split(F):
    counter += 1
    print('fold:', counter)
    X_train, X_test = F[train_index], F[test_index]
    y_train, y_test = target[train_index], target[test_index]
    
    fm_mdl = FactorizationMachine(step=0.9, k=3)
    fm_mdl.fit(X_train, y_train, n_epoch=5, batch_size=1024)

    predictions = fm_mdl.predict(X_train).reshape(-1)
    RMSE_history_tr.append(RMSE(predictions, y_train))
    R_2_history_tr.append(R_2(predictions, y_train))
    
    predictions = fm_mdl.predict(X_test).reshape(-1)
    RMSE_history_ts.append(RMSE(predictions, y_test))
    R_2_history_ts.append(R_2(predictions, y_test))

KFold(n_splits=3, random_state=None, shuffle=False)
fold: 1
fold: 2
fold: 3


In [8]:
metricsNames = ['R2-train','R2-test', 'RMSE-train', 'RMSE-test']
colNames = ['T1', 'T2', 'T3']
report = pd.DataFrame(np.vstack([R_2_history_tr, R_2_history_ts, RMSE_history_tr, RMSE_history_ts]), index=metricsNames, 
                     columns=colNames)
E = pd.Series(report.mean(axis=1), index = report.index)
STD = pd.Series(report.std(axis=1), index = report.index)
report['E'] = E
report['STD'] = STD

report

,T1,T2,T3,E,STD
R2-train,0.190290,0.233542,0.236887,0.220240,0.025991
R2-test,0.121940,0.160876,0.088738,0.123851,0.036107
RMSE-train,0.960427,0.964683,0.948070,0.957727,0.008629
RMSE-test,1.039181,0.962953,1.033929,1.012021,0.042575
